following are the target:
   1. Implement pipeline
   2. implement staking
   3. use knn for imputation

In [5]:
import pandas as pd 
import sklearn
import numpy as np
from sklearn.model_selection import GridSearchCV

In [3]:
import math

In [ ]:
#def model_eval_reg(model)
    

columns=[]
model_fs=XGBClassifier()
kfold=StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
for i in X_train.columns:
    columns.append(i)
    X_Sel=X_train.loc[:,columns]
    cv_results=cross_val_score(model_fs,X_Sel,y_train,cv=kfold,scoring='accuracy')
    #print('mean: { 0}    std: {1}'.format(cv_results.mean(),cv_results.std()))
    print(' %f (%f)' % ( cv_results.mean(), cv_results.std()))


In [ ]:

models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
#RandomForest params
max_feature=round(math.sqrt(len(X_train.columns)))
min_sample_leaf_value=50
models.append(('RandomForest',RandomForest(max_features=max_feature,min_sample_leaf=min_sample_leaf_value,oob_score=True)))
#XGB params

models.append(('xgboost',XGBClassifier()))
# evaluate each model in turn
results = []
names = []
kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    #print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    #print(f'name:{name}   result_mean={cv_results.mean()}')
    print('{0}     {1:.2f}'.format(name,cv_results.mean()))

In [2]:
def classification_model_evaluation(models,X_train,y_train,pcv=10,score='accuracy'):
    result={}
    for name,model in models:
        cv_result=cross_val_score(model,X_train,y_train,cv=pcv,scoring=score)
        result[name]=cv_result
        print('{0}     {1:.2f}'.format(name,cv_result.mean()))
        
    

In [1]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
rfc=RandomForestClassifier(max_features=sqrt(n_features),mean_sample_leaf=50,oob_score=True)

In [4]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=0,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Empirical good default values are max_features=n_features for regression problems, and max_features=sqrt(n_features) for classification tasks.usually max_features=n_features/3 is good default for regression problems
https://blog.datadive.net/selecting-good-features-part-iii-random-forests/
https://people.eecs.berkeley.edu/~jrs/189s17/lec/16.pdf
if possible control max_depth by min_sample_leaf parameter
min_samples_split need not be checked .Its default of 2 is good.


In [ ]:

#depth_val=np.linspace(4,64,5,endpoint=True)
depth_val=[4,8,16,32]
#num_of_estmators=[10,20,30]
min_samples_leaf_val=[20,40,60,80,100]
max_features_val=[3,2,1]
#DecisionTree_param={'max_depth':depth,max_feature=max_feature_sel,min_sample_leaf=min_sample_leaf_val}
#np.arange is also available

In [ ]:
def GridSearchRF(X_train,y_train):
    RandomForest_Param={'max_depth':depth_val,'min_samples_leaf':min_samples_leaf_val,'max_features':max_features_val}
    rfc=RandomForestClassifier()
    grid_search=GridSearchCV(estimator=rfc,param_grid=RandomForest_Param,cv=10)
    grid_search.fit(X_train,y_train)
    print(grid_search.best_params_)
    model=RandomForestClassifier(max_depth=grid_search.best_params_['max_depth'],max_features=grid_search.best_params_['max_features'],min_samples_leaf=grid_search.best_params_['min_samples_leaf'])
    return model

https://machinelearningmastery.com/configure-gradient-boosting-algorithm/
https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/
https://www.linkedin.com/pulse/approaching-almost-any-machine-learning-problem-abhishek-thakur/
## gradient boosting machine tuning
tree specific parameters:
-------------choose a high learning rate and find optimum number of trees for that tree
-------------Tune tree specific parameter
-------------change the learning rate and optimum number of  trees

for base model: min_samples_split = 500 ,min_samples_leaf = 50,max_depth = 8 ,max_features = ‘sqrt’ ,subsample = 0.8
tree Based parameters fro GBM:
1.Tune max_depth and min_samples_split
2.Tune min_samples_leaf
3.Tune max_features
   
   -----------------
   use early stopping :
   validation_fraction=.2
   n_iter_no_change=
   tol
   
   return values:
   feature_importances_
   
   
   
   ------------------------------------------------------
   learning_rate=0.1 (shrinkage).
n_estimators=100 (number of trees).
max_depth=3.
min_samples_split=2.
min_samples_leaf=1.
subsample=1.0.

## XGBoost tuning
 tree specific parameters: max_depth, min_child_weight, gamma, subsample, colsample_bytree,scale_pos_weight(because of class imbalance)
four steps of xgb parameter tuning:
    1.choose a high learning rate and  find the optimum number of trees for this learning rate
    2.tune tree specific parameter : max_depth,min_child_weight,gamma,subsample,colsample_bytree
    3. tune regularization parameters :lambda,alpha
    4.lower the learning rate;

In [ ]:
tree based parameters: max_depth = 5,min_child_weight = 1,gamma = 0,subsample, colsample_bytree = 0.8,scale_pos_weight = 1
uning Regularization Parameters: reg_alpha,reg_lambda

In [4]:
from xgboost import XGBClassifier

In [5]:
xgbc=XGBClassifier(objective='multi:softmax',booster='gbtree',n_estimators=100,silent=0,learning_rate=.3,min_child_weight=1,max_depth=6,gamma=0,max_delta_setp=0,colsample_bytree=1)

In [6]:
xgbc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.3, max_delta_setp=0, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softmax', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=0, subsample=1, verbosity=1)

In [5]:
param=xgbc.get_xgb_params

In [6]:
xgbc.

<bound method XGBModel.get_xgb_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.3, max_delta_setp=0, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softmax', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=0, subsample=1, verbosity=1)>

In [7]:
learning_rate_val=[0.01,0.02,0.04,0.08,.1,.2,.4,.8] #default .3
subsample_val=[.5,.6,.8,1]
colsample_bytree_val=[.5,.7,.9,1]
n_estimators_val=[50,100,200]
#min_child_weight_val=[1,2,3,4]
#objective_val=['binary:logistic','multi:softmax','multi:softprob']

In [9]:
def GridSearchXGBC(X_train,y_train,lr=.3,trees=50):
    XGB_classifier_param={'learning_rate':learning_rate_val,'subsample':subsample_val,'colsample_bytree':colsample_bytree_val}
    xgbc=XGBClassifier(objective='multi:softmax',booster='gbtree',n_estimators=100,silent=0,learning_rate=.3,min_child_weight=1,max_depth=6,gamma=0,max_delta_setp=0,colsample_bytree=1)
    gs=GridSearchCV(estimator=xgbc,param_grid=XGB_classifier_param,cv=10)
    gs.fit(X_train,y_train)
    print(gs.best_params_)
    model=XGBClassifier(objective='multi:softmax',booster='gbtree',n_estimators=100,silent=0,learning_rate=gs.best_params_['learning_rate'],subsample=gs.best_params_['subsample_val'] ,min_child_weight=1,max_depth=6,gamma=0,max_delta_setp=0,colsample_bytree=gs.best_params_['cosample_bytree'])

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def GridSearchGBM(X_train,y_train,lr=.3,trees=50):
    default_learning_rate=.3
    default_num_of_tree=50
    base_GBM=GradientBoostingClassifier(learning_rate=lr,n_estimators=trees,max_depth=6,max_features='sqrt',min_samples_split=50,min_samples_leaf=10,subsample=1.0,n_iter_no_change=20,validation_fraction=.2)
    max_depth_val=[10,30,50,70,90,20]
    #max_depth_val=[10]
    min_samples_split_val=[4,8,16,32,64,128,256,512]
    #min_samples_split_val=[4]
    n_fe=len(X_Sel.columns)
    max_features_val=[n_fe//2,n_fe//3,round(math.sqrt(n_fe)),n_fe]
    tol=1e-4
    first_tuning={'max_depth':max_depth_val,'min_samples_split':min_samples_split_val,'max_features':max_features_val}
    grid_model=GridSearchCV(estimator=base_GBM,param_grid=first_tuning,cv=10)
    grid_model.fit(X_train,y_train)
    #learning-rate=2-10/num of trees
    #learning_rate_val=[.2,.1,.05,.02,.01]
    learning_rate_val=[.2,.1]
    #n_estimators_val=[50,100,200,400,500,800,1000]
    n_estimators_val=[100,200]
    second_tuning={'learning_rate':learning_rate_val,'n_estimators':n_estimators_val}
    final_model=GridSearchCV(estimator=grid_model.best_estimator_,param_grid=second_tuning,cv=10)
    return final_model

In [8]:
gb=GradientBoostingClassifier()

In [9]:
gb


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [15]:
base_GBM=GradientBoostingClassifier(learning_rate=.3,n_estimators=50,max_depth=6,max_features='sqrt',min_samples_split=50,min_samples_leaf=10,subsample=1.0)

In [ ]:
def GridSearchXGBC(X_train,y_train,lr=.3,trees=50):
    #XGB_classifier_param={'learning_rate':learning_rate_val,'subsample':subsample_val,'colsample_bytree':colsample_bytree_val}
    base_XGBC=XGBClassifier(objective='multi:softmax',booster='gbtree',n_estimators=trees,silent=0,learning_rate=lr,min_child_weight=1,max_depth=6,gamma=0,max_delta_setp=0,colsample_bytree=1)
    max_depth_val=[100]
    #max_depth_val=[10,30,50,70,90,20]
    min_child-weight_val=[1]
    #min_child_weight_val=[1,2,4,8] #equivalent of min_child_leaf in GBM
    #eta=learning_rate
    first_tuning={'max_depth':max_depth_val,'min_child_weight':min_child_weight_val}
    grid_model=GridSearchCV(estimator=base_XGBC,param_grid=first_tuning,cv=10)
    grid_model.fit(X_train,y_train)
    #learning-rate=2-10/num of trees
    #learning_rate_val=[.2,.1,.05,.02,.01]
    learning_rate_val=[.2,.1]
    #n_estimators_val=[50,100,200,400,500,800,1000]
    n_estimators_val=[100,200]
    second_tuning={'learning_rate':learning_rate_val,'n_estimators':n_estimators_val}
    final_model=GridSearchCV(estimator=grid_model.best_estimator_,param_grid=second_tuning,cv=10)
    return final_model
     
    

In [ ]:
    gs=GridSearchCV(estimator=xgbc,param_grid=XGB_classifier_param,cv=10)
    gs.fit(X_train,y_train)
    print(gs.best_params_)
    model=XGBClassifier(objective='multi:softmax',booster='gbtree',n_estimators=100,silent=0,learning_rate=gs.best_params_['learning_rate'],subsample=gs.best_params_['subsample_val'] ,min_child_weight=1,max_depth=6,gamma=0,max_delta_setp=0,colsample_bytree=gs.best_params_['cosample_bytree'])

In [ ]:
def ensemble(models,layers,X_train,y_train):
    for index in np.arange(0,layers,1):
        model=models[index]
        X=X_train[index]
        y=y_train[index]
        X_test=X_train[index+1]
        for m in models:
            m.fit(X,y)
            input=m.predict(X_test)
            

In [3]:
def ensemble_level_train(base_models,X_train,X_holdout,y_train,y_holdout):
    intermediate_input=pd.DataFrame()
    for model in base_models:
        model.fit(X_train,y_train)
        y_temp=model.predict(X_holdout)
        intermediate_input[model.__class__.__name__]=y_temp;
        print(model.__class__.__name__,accuracy_score(y_holdout,y_temp))
    return intermediate_input
    
    
        
        

In [4]:
def ensemble_level_predict(base_models,X_test):
    intermediate_input=pd.DataFrame()
    for model in base_models:
        y_temp=model.predict(X_test)
        intermediate_input[model.__class__.__name__]=y_temp;
    return intermediate_input
        

In [2]:
for index in np.arange(0,3,1):
    print(index)

0
1
2


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


Important hyperparam of random forest in decreasing order: <br>
1.n_estimators <br>
2.max_features <br>
3.max_depth
4.min_samples_split 
5.min_samples_leaf 
-- BootStrap #need to learn..ne


In [ ]:
def GridSearchRandomForest(X_train,y_train,trees=50):
    base_random_forest=RandomForestClassifier(n_estimators=trees,random_state=108 )
    #max_depth_val=[10,30,50,70,100]
    max_depth_val=[10]
    #min_samples_split_val=[2,4,8,12,16]
    min_samples_split_val=[4]
    #min_samples_leaf_val=[2,4,6,8,10]
    min_samples_leaf_val=[2]
    first_tuning={"max_depth":max_depth_val,"min_samples_split":min_samples_split_val,"min_samples_leaf":min_samples_leaf_val}
    model1=GridSearchCV(estimator=base_random_forest,param_grid=first_tuning,cv=10)
    model1.fit(X_train,y_train)
    #n_estimators_val=[100,200,400,600]
    n_estimators_val=[100]
    max_features_val=[.5,1.0]
    second_tuning={"n_estimators":n_estimators_val,"max_features":max_features_val}
    final_model=GridSearchCV(estimator=model1.best_estimator_,param_grid=second_tuning,cv=10)
    final_model.fit(X_train,y_train)
    return final_model

In [ ]:
def RandomSearchRandomForest(X_train,y_train,trees=50):
    n_estimators_val = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_features_val = ['auto', 'sqrt'] #[.2,.4,.6,.8]
    max_depth_val = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth_val.append(None)
    min_samples_split_val = [2, 5, 10]
    min_samples_leaf_val = [1, 2, 4]
    bootstrap_val = [True, False]
    random_grid={'n_estimators':n_estimators_val,'max_features':max_features_val,'max_depth':max_depth_val,
                 'min_samples_split':min_samples_split_val,
                 'min_samples_leaf':min_samples_leaf_val,
                 'bootstrap':bootstrap_val
                }
    base_model=RandomForestClassifier(random_state=108)
    random_model=RandomizedSearchCV(estimator=base_model,param_distributions=random_grid,n_iter=100,cv=10,random_state=108)
    random_model.fit(X_train,y_train)
    return random_model